# Import Relevant Packages

In [ ]:
import praw, openpyxl, pandas as pd, re, requests

# Initializing praw

> Fields to be filled in: `client_id, client_secret, password, user_agent, username`
> 
> More details can be found at https://praw.readthedocs.io/en/stable/getting_started/quick_start.html

In [ ]:
reddit = praw.Reddit(
    client_id="client_id",
    client_secret="client_secret",
    password="password",
    user_agent="user_agent",
    username="username",
) 
print(reddit.user.me())

# Class to Extract Comments from Reddit

> API Keys need to be filled in [`api_key`]

In [ ]:
class polarity_classification:
    wb = None
    def __init__(self, name):
        self.name = name
        self.FILENAME = f"{self.name}.xlsx"
        self.LANG = 'en' #Language of API to be used
        self.APIKEY = ''
        self.APIURL = ''

    def open_or_create_workbook(self):
        try:
            self.wb = openpyxl.load_workbook(filename = self.FILENAME)
            print('Opened workbook')
        except:
            self.wb = openpyxl.Workbook()
            print('Create new workbook')
            print(self.wb.sheetnames)
            ws = self.wb.active
            ws.title = f"Unprocessed Comments"
            heading = [['Comment', 'Score', 'Author', 'Comment ID']]
            for row in heading:
                ws.append(row)
            self.wb.save(self.FILENAME)
            print(self.wb.path)

    def close_workbook(self):
        self.wb.save(self.FILENAME)
        self.wb.close()

    def get_api_ws(self, api_type):
        if api_type == 'concept_parsing':
            api_key = ''
            ws_name = 'Concept Parsing'
        elif api_type == 'subjectivity_detection':
            api_key = ''
            ws_name = 'Subjectivity Detection'
        elif api_type == 'polarity_classification':
            api_key = ''
            ws_name = 'Polarity Classification'
        elif api_type == 'intensity_ranking':
            api_key = ''
            ws_name = 'Intensity Ranking'
        elif api_type == 'emotion_recognition':
            api_key = ''
            ws_name = 'Emotion Recognition'
        elif api_type == 'aspect_extraction':
            api_key = ''
            ws_name = 'Aspect Extraction'
        elif api_type == 'personality_prediction':
            api_key = ''
            ws_name = 'Personality Prediction'
        elif api_type == 'sarcasm_identification':
            api_key = ''
            ws_name = 'Sarcasm Detection'
        elif api_type == 'depression_categorization':
            api_key = ''
            ws_name = 'Depression Categorization'
        elif api_type == 'toxicity_spotting':
            api_key = ''
            ws_name = 'Toxicity Spotting'
        elif api_type == 'engagement_measurement':
            api_key = ''
            ws_name = 'Engagement Measurement'
        elif api_type == 'wellbeing_assessment':
            api_key = ''
            ws_name = 'Well-Being Assessment'
        else:
            api_key = None
            ws_name = None
        self.APIKEY = api_key
        self.APIURL = f'https://sentic.net/api/{self.LANG}/{self.APIKEY}.py?text='
        ws = self.open_or_copy_worksheet(ws_name)
        ws.cell(row= 1, column = 5, value = ws_name)

        return self.open_or_copy_worksheet(ws_name), ws_name

    def extract_comments(self, url):
        ws_name = 'Unprocessed Comments'
        
        api_df = pd.read_excel(self.FILENAME, sheet_name=ws_name)
        df = pd.DataFrame({'Comment': [], 'Score': [],'Author': [],'Comment ID': []})
        submission = reddit.submission(url=url)
        bot_list = ['AmputatorBot', 'XGramatik-Bot', 'RemindMeBot']
        single_word, deleted_comments, removed_comments, duplicates, gif_count, url_count, total = 0, 0, 0, 0, 0, 0, 0
        bot_comments = 0
        comment_count, comment_after_deduplication = 0,0
        
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            total+=1
            commentor = comment.author
            if commentor:
                username = f'{commentor.name}'
            else:
                username = ''
            if username in bot_list:
                bot_comments += 1
                continue
            existing_comment = api_df['Comment ID'] == comment.id
            if existing_comment.any():
                continue
                
            text = comment.body

            if text == '[deleted]':
                deleted_comments += 1
                continue
            elif text == '[removed]':
                removed_comments += 1
                continue

            count = len(re.findall(r'\w+', text))
            if count<=1:
                single_word+=1
                continue

            gif_pattern = r"!\[gif\]\(giphy\|[A-Za-z0-9]+(?:\|[A-Za-z0-9]+)*\)"
            gifs = re.findall(gif_pattern, text, flags=re.M)
            url_pattern = url_pattern = r'https?://\S+'
            urls = re.findall(url_pattern, text, flags=re.M)
            if urls:
                text = re.sub(url_pattern, '', text, flags=re.M)
                count = len(re.findall(r'\w+', text))
                if count<=1:
                    single_word+=1
                    continue
                url_count+=1
            if gifs:
                text = re.sub(gif_pattern, '', text, flags=re.M)
                count = len(re.findall(r'\w+', text))
                if count<=1:
                    single_word+=1
                    continue
                gif_count+=1

            text = re.sub(r"[;&#{}]", ":", text)

            new_row = pd.DataFrame({'Comment': [text], 'Score': [comment.score],'Author': [username],'Comment ID': [comment.id]})

            df = pd.concat([df, new_row], ignore_index=True)
            comment_count += 1
        
        df = df.drop_duplicates(subset=['Comment'], keep='first')
        
        with pd.ExcelWriter(self.FILENAME, engine='openpyxl', mode='a',if_sheet_exists='overlay') as writer:
            book = writer.book
            if ws_name in book.sheetnames:
                sheet = book[ws_name]
                startrow = sheet.max_row
            else:
                startrow = 0
            df.to_excel(writer, index=False, sheet_name=ws_name, startrow=startrow, header=False if startrow > 0 else True)

            
        print(f'comment_count: {comment_count}')
        print(f'comment_count without duplicates: {len(df.index)}')
        print(f'bot_comments: {bot_comments}')     
        print(f'duplicates: {duplicates}')
        print(f'single_word: {single_word}')
        print(f'deleted_comments: {deleted_comments}')
        print(f'removed_comments: {removed_comments}')
        print(f'gif_count: {gif_count}')
        print(f'url_count: {url_count}')
        print(f'total: {total}')

    def run_api(self, api_type):
        ws, ws_name = self.get_api_ws(api_type)
        self.APIURL = f'https://sentic.net/api/en/{self.APIKEY}.py?text='
        self.close_workbook()
        api_df = pd.read_excel(self.FILENAME, sheet_name=ws_name)
        
        api_df = self.api_request(api_df)

        if ws_name in self.wb.sheetnames:
            sheet = self.wb[ws_name]
            self.wb.remove(sheet)
            self.close_workbook()
        
        with pd.ExcelWriter(self.FILENAME, engine='openpyxl', mode='a') as writer:
            api_df.to_excel(writer, index=False, sheet_name=ws_name)\
                
    def api_request(self, api_df):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
        count = 1
        total_rows = api_df.shape[0]
        for index, row in api_df.iterrows():
            clear_output(wait=True)
            if pd.isna(api_df.iloc[index, 4]):
                text = api_df.at[index, 'Comment']
                # print(text)
                label = str(requests.get(self.APIURL + str(text), headers=headers).content)[2:-3]
                # print(label)
                api_df.iloc[index, 4] = label
                print(f'{count}/{total_rows} Comment ID: {api_df.at[index, "Comment ID"]} Label: {api_df.iloc[index, 4]}')
            count+=1
        return api_df
    
    def open_or_copy_worksheet(self, worksheet_name):
        try:
            if self.wb[worksheet_name]:
                return self.wb[worksheet_name]
        except:
            api_ws = self.wb.copy_worksheet(self.wb["Unprocessed Comments"])
            api_ws.title = worksheet_name
            return api_ws

# Creating Object and Extracting Reddit Comments

> Fields to be filled in: `reddit_url, object_name, excel_workbook`

In [ ]:
url = "reddit_url"
object_name = polarity_classification("excel_workbook")
object_name.open_or_create_workbook()
object_name.extract_comments(url)

# Running API

> Fields to be filled in: `object_name, api_name`

In [ ]:
object_name.run_api('api_name')

# End of Notebook